In [82]:
import zipfile
import os
import pandas as pd
import numpy as np


In [83]:
def flow_matrix(path_in, out_file, filter_column=None, unzip=False):
    if unzip:
        with zipfile.ZipFile(path_in, 'r') as zip_ref:
            zip_ref.extractall()
    df_count = pd.DataFrame()
    if filter_column is None:
        cols = ['start_region','dest_region']
    else:
        cols = ['start_region','dest_region', filter_column]
    for path in os.listdir(path_in[:-4]):
        if 'travels_results' in path:
            df = pd.read_csv(path_in[:-4] +"/" +path)
            df_count = pd.concat([df_count, df.groupby(cols).size()
            .reset_index(name='counts')])
    df_count = df_count.groupby(cols).agg({'counts': ['mean', 'std']})
    df_count.columns = df_count.columns.droplevel(0)
    df_count = df_count.reset_index(level=[0,1])
    df_count.to_pickle(out_file)
    return df_count
    

In [85]:
def flow_matrix_join(path_in, out_file, filter_column, unzip=False):
    if unzip:
        with zipfile.ZipFile(path_in, 'r') as zip_ref:
            zip_ref.extractall()
    df_count = pd.DataFrame()
    cols = ['start_region','dest_region', filter_column]
    for path in os.listdir(path_in[:-4]):
        if 'travels_results' in path:
            df = pd.read_csv(path_in[:-4] +"/" +path)
            path_end = path.split("_")
            df_agents = pd.read_csv(path_in[:-4] +"/" + "agents_results_"+ path_end[-1])
            df_merged = df.join(df_agents, on='agent_id', rsuffix="_agent")
            df_count = pd.concat([df_count, df_merged.groupby(cols).size()
            .reset_index(name='counts')])
    df_count = df_count.groupby(cols).agg({'counts': ['mean', 'std']})
    df_count.columns = df_count.columns.droplevel(0)
    df_count = df_count.reset_index(level=[0,1])
    df_count.to_pickle(out_file)
    return df_count

In [92]:
flow_matrix('scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0.zip', 'fm_all_sceanario_0_0_0.pkl', unzip=True)

,start_region,dest_region,mean,std
0,1,1,408.170000,28.003266
1,1,2,93.210000,8.459404
2,1,3,56.100000,7.521686
3,1,4,40.950000,6.104105
4,1,5,67.090000,8.656282
...,...,...,...,...
139772,454,404,1.200000,0.404520
139773,454,451,4.181818,1.740066
139774,454,452,1.800000,1.060213
139775,454,453,2.534091,1.312617


In [93]:
flow_matrix_join('scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0.zip', 'fm_asc_sceanario_0_0_0.pkl', 'age_sex')

,start_region,dest_region,mean,std
age_sex,,,,
16-19_K,1,1,2.613333,1.746760
16-19_M,1,1,2.896552,1.738982
20-24_K,1,1,7.400000,3.058355
20-24_M,1,1,6.071429,2.788969
25-44_K,1,1,95.310000,13.230432
...,...,...,...,...
45-65_M,454,454,6.220000,2.750868
6-15_K,454,454,2.625000,1.283015
6-15_M,454,454,1.760563,1.176736


In [94]:
flow_matrix('scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0.zip', 'fm_tm_sceanario_0_0_0.pkl', filter_column ='transport_mode')

,start_region,dest_region,mean,std
transport_mode,,,,
0,1,1,82.150000,13.464037
1,1,1,30.280000,7.547934
2,1,1,271.260000,22.861295
3,1,1,24.480000,7.812810
0,1,2,25.510000,5.215546
...,...,...,...,...
3,454,453,1.071429,0.267261
0,454,454,9.960000,3.706178
1,454,454,3.916667,2.282507


In [62]:
counts

,start_region,dest_region,mean,std
0,1,1,273.940000,21.033899
1,1,2,61.460000,6.827944
2,1,3,38.820000,6.775376
3,1,4,27.900000,5.334715
4,1,5,45.000000,6.207303
...,...,...,...,...
137962,454,404,1.133333,0.351866
137963,454,451,3.326087,1.898003
137964,454,452,1.428571,0.654654
137965,454,453,2.000000,1.309307


In [78]:
counts2 = flow_matrix_filter('scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0.zip', 'scenarios_1_1', 'age_sex')


       'bicycle_infrastr_comfort', 'pedestrian_inconvenience',
       'household_persons', 'household_cars', 'household_bicycles',
       'travels_num'],
      dtype='object')
Index(['Unnamed: 0', 'agent_id', 'start_region', 'start_place_type',
       'dest_region', 'dest_place_type', 'travel_start_time', 'transport_mode',
       'is_driver', 'Step', 'AgentID', 'agent_id_agent', 'home_region',
       'age_sex', 'pub_trans_comfort', 'pub_trans_punctuality',
       'bicycle_infrastr_comfort', 'pedestrian_inconvenience',
       'household_persons', 'household_cars', 'household_bicycles',
       'travels_num'],
      dtype='object')
Index(['Unnamed: 0', 'agent_id', 'start_region', 'start_place_type',
       'dest_region', 'dest_place_type', 'travel_start_time', 'transport_mode',
       'is_driver', 'Step', 'AgentID', 'agent_id_agent', 'home_region',
       'age_sex', 'pub_trans_comfort', 'pub_trans_punctuality',
       'bicycle_infrastr_comfort', 'pedestrian_inconvenience',
       'househ

In [108]:
df = pd.read_pickle('fm_tm_sceanario_0_0_0.pkl')
df = df.reset_index()
#df = df.reset_index(level=0, inplace=True)

In [111]:
df.columns

Index(['transport_mode', 'start_region', 'dest_region', 'mean', 'std'], dtype='object')

In [57]:
x

,start_region,dest_region,mean,std
0,1,1,273.940000,21.033899
1,1,2,61.460000,6.827944
2,1,3,38.820000,6.775376
3,1,4,27.900000,5.334715
4,1,5,45.000000,6.207303
...,...,...,...,...
137962,454,404,1.133333,0.351866
137963,454,451,3.326087,1.898003
137964,454,452,1.428571,0.654654
137965,454,453,2.000000,1.309307


In [50]:
x

,mean,std
dest_region,,
1,273.940000,21.033899
2,61.460000,6.827944
3,38.820000,6.775376
4,27.900000,5.334715
5,45.000000,6.207303
...,...,...
404,1.133333,0.351866
451,3.326087,1.898003
452,1.428571,0.654654


In [38]:
counts.loc[(counts['start_region']==1) & (counts['dest_region']==1)]

,start_region,dest_region,counts
0,1,1,244
0,1,1,248
0,1,1,286
0,1,1,268
0,1,1,283
0,1,1,261
0,1,1,275
0,1,1,254
0,1,1,306
0,1,1,281


In [8]:
os.listdir("scenarios_1_1")

['scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0']